# Dán nhãn

## 1. Sơ khai

In [ ]:
import pandas as pd
import re

def clean_text(text):
    if text.count(";") >= 2:
        text = ";".join(text.split(";")[:2])
    text = re.split(r'\s*-\s*', text)[0]
    return text.strip()

def categorize(df, keywords_list, category_mapping):
    df["Label"] = None  # Khởi tạo cột nhãn
    keywords_dict = {kw.lower(): label for label, keywords in category_mapping.items() for kw in keywords}

    pattern = "|".join(map(re.escape, keywords_dict.keys()))
    mask = df["Product List"].str.lower().str.extract(f'({pattern})', expand=False)

    df["Label"] = mask.map(keywords_dict)
    return df

def process_data(file_path, fa_keywords, fa_mapping, c_keywords, c_mapping):
    df = pd.read_csv(file_path, encoding="utf-8")
    df.drop(['Label'], axis=1, inplace=True, errors='ignore')
    df['Left_Text'] = df['Product List'].apply(clean_text)

    # Gán nhãn theo nhóm Fa
    df = categorize(df, fa_keywords, fa_mapping)

    # Lọc những sản phẩm chưa được gán nhãn
    df_child = df[df['Label'].isna()].copy()

    # Gán nhãn theo nhóm C
    df_child = categorize(df_child, c_keywords, c_mapping)

    # Gộp kết quả
    df_result = df.merge(df_child, on="Product List", how="left", suffixes=('_df', '_df_child'))
    df_result["Label"] = df_result["Label_df"].combine_first(df_result["Label_df_child"])
    df_result = df_result[['Product List', 'Label']]

    # Lưu kết quả
    df_result.to_excel("df_final_result.xlsx", index=False, engine="openpyxl")

    # Xuất những sản phẩm chưa có nhãn
    df_remain = df_result[df_result['Label'].isna()]
    df_remain.to_excel("df_remain.xlsx", index=False, engine="openpyxl")

    return df_result, df_remain

# Danh sách từ khóa
fa_keywords = ['Áo', 'Quần', 'Vải', 'Chỉ', 'Khoá', 'Sợi', 'Xơ']
fa_mapping = {
    'Quần/Áo': ['Áo', 'Quần'],
    'Vải': ['Vải'],
    'Phụ_trợ': ['Chỉ', 'Khoá'],
    'Sợi': ['Sợi'],
    'Nguyên_liệu': ['Xơ']
}

c_keywords = ['jacket', 'fabric', 'thoi', 'woven', 'knitted', 'nlwoven', 'coated', 'phủ', 'lining', 'sheet',
              'feather', 'main', 'fiber', 'staple', 'chất', 'plastic', 'metal', 'kéo', 'khóa', 'zipper', 'thread',
              'đinh', 'filament', 'tổng_hợp', 'synthetic', 'yarn', 'bag', 'túi']
c_mapping = {
    'Quần/Áo': ['jacket'],
    'Vải': ['fabric', 'thoi', 'woven', 'knitted', 'nlwoven', 'coated', 'phủ', 'lining', 'sheet', 'feather', 'main'],
    'Nguyên_liệu': ['fiber', 'staple', 'chất', 'plastic', 'metal'],
    'Phụ_trợ': ['kéo', 'khóa', 'zipper', 'thread', 'đinh'],
    'Sợi': ['filament', 'tổng_hợp', 'synthetic', 'yarn'],
    'Túi_xách': ['bag', 'túi']
}

# Chạy xử lý
final_df, remain_df = process_data("Product.csv", fa_keywords, fa_mapping, c_keywords, c_mapping)
print(final_df.head())
print("Unlabeled rows saved in df_remain.xlsx")


                                 Product List    Label
0   Chỉ may 100D/3 COLOR (3000 met/cone) #&VN  Phụ_trợ
1     Chỉ may 150D COLOR (5000 met/cone) #&VN  Phụ_trợ
2     Chỉ may 150D WHITE (5000 met/cone) #&VN  Phụ_trợ
3     Chỉ may 20/2 COLOR (3000 met/cone) #&VN  Phụ_trợ
4    Chỉ may 20/3 color  ( 2000 mét/cuộn)#&VN  Phụ_trợ
Unlabeled rows saved in df_remain.xlsx


## 2. Code oke

In [ ]:
import pandas as pd
import re

def clean_text(text):
    if text.count(";") >= 2:
        text = ";".join(text.split(";")[:2])
    text = re.split(r'\s*-\s*', text)[0]
    return text.strip()

def categorize(df, keywords_list, category_mapping):
    df["Label"] = None  # Khởi tạo cột nhãn
    keywords_dict = {kw.lower(): label for label, keywords in category_mapping.items() for kw in keywords}

    pattern = "|".join(map(re.escape, keywords_dict.keys()))
    mask = df["Product List"].str.lower().str.extract(f'({pattern})', expand=False)

    df["Label"] = mask.map(keywords_dict)
    return df

def process_data(file_path, fa_keywords, fa_mapping, c_keywords, c_mapping):
    df = pd.read_csv(file_path, encoding="utf-8")
    df.drop(['Label'], axis=1, inplace=True, errors='ignore')
    df['Left_Text'] = df['Product List'].apply(clean_text)

    # Gán nhãn theo nhóm Fa
    df = categorize(df, fa_keywords, fa_mapping)

    # Lọc những sản phẩm chưa được gán nhãn
    df_child = df[df['Label'].isna()].copy()

    # Gán nhãn theo nhóm C
    df_child = categorize(df_child, c_keywords, c_mapping)

    # Gộp kết quả
    df_result = df.merge(df_child, on="Product List", how="left", suffixes=('_df', '_df_child'))
    df_result["Label"] = df_result["Label_df"].combine_first(df_result["Label_df_child"])
    df_result = df_result[['Product List', 'Label']]

    # Lưu kết quả
    df_result.to_excel("df_final_result.xlsx", index=False, engine="openpyxl")

    # Xuất những sản phẩm chưa có nhãn
    df_remain = df_result[df_result['Label'].isna()]
    df_remain.to_excel("df_remain.xlsx", index=False, engine="openpyxl")

    return df_result, df_remain

# Danh sách từ khóa
fa_keywords = ['kg','Áo', 'Quần', 'vải','Chỉ', 'Khoá', 'Sợi', 'Xơ','short','shirt','coat','vest','gile','blazer',
               'bông','thun','knit','textile','wool', 'sợi','Accessorie',
               'ring','cúc','chốt','nhãn','thẻ','card','đạn','dây','chun','band','wire','paper','giấy','construction','dựng','bo','móc',
               'băng','glue','button','sew','vòng','keo','bao','nút','assort','rivet','stopper','miếng','badges','rope','hide','khuy','nẹp','tag',
               'mác','tape','ô zê','decal','đệm','conduit','hanger','fur','ball','kẹp','thun','hook','lông','decora','gòn','nhân tạo','da','label',
               'hộp','lock','mắc','chuông','leather']
fa_mapping = {
    'Quần/Áo': ['Áo', 'Quần', 'short','shirt','coat','vest','gile','blazer'],
    'Vải': ['vải','bông','thun','knit','textile'],
    'Sợi': ['wool','sợi'],
    'Nguyên_liệu': ['Xơ','kg','gòn','nhân tạo','da'],
    'Phụ_trợ': ['Chỉ', 'Khoá','Accessorie','ring','cúc','chốt','nhãn','thẻ','card','đạn','dây','chun','band','wire','paper','giấy','construction','dựng','bo','móc',
              'băng','glue','button','sew','vòng','keo','bao','nút','assort','rivet','stopper','miếng','badges','rope','hide','khuy','nẹp','tag',
              'mác','tape','ô zê','decal','đệm','conduit','hanger','fur','ball','kẹp','thun','hook','lông','decora','label','hộp','lock','mắc','chuông','leather']

}

c_keywords = ['polyester','cotton','t-shirts','cloth','mũ','oze','bolen','đê','thuộc','jacket', 'fabric', 'thoi', 'woven', 'knitted', 'nlwoven', 'coated', 'phủ', 'lining', 'sheet',
              'feather', 'main', 'fiber', 'staple', 'chất', 'plastic', 'metal', 'kéo', 'khóa', 'zipper', 'thread',
              'đinh', 'filament', 'tổng_hợp', 'synthetic', 'yarn', 'bag', 'túi','chặn']
c_mapping = {
    'Vải': ['fabric', 'thoi', 'woven', 'knitted', 'nlwoven', 'coated', 'phủ', 'lining', 'sheet', 'feather', 'main','polyester','cotton'],
    'Nguyên_liệu': ['fiber', 'staple', 'chất', 'plastic', 'metal','thuộc'],
    'Quần/Áo': ['jacket','t-shirts','cloth','mũ'],
    'Phụ_trợ': ['kéo', 'khóa', 'zipper', 'thread', 'đinh','oze','bolen','đê','chặn'],
    'Sợi': ['filament', 'tổng_hợp', 'synthetic', 'yarn'],
    'Túi_xách': ['bag', 'túi']
}

# Chạy xử lý
final_df, remain_df = process_data("Product.csv", fa_keywords, fa_mapping, c_keywords, c_mapping)
print(final_df.head())
print("Unlabeled rows saved in df_remain.xlsx")


                                 Product List    Label
0   Chỉ may 100D/3 COLOR (3000 met/cone) #&VN  Phụ_trợ
1     Chỉ may 150D COLOR (5000 met/cone) #&VN  Phụ_trợ
2     Chỉ may 150D WHITE (5000 met/cone) #&VN  Phụ_trợ
3     Chỉ may 20/2 COLOR (3000 met/cone) #&VN  Phụ_trợ
4    Chỉ may 20/3 color  ( 2000 mét/cuộn)#&VN  Phụ_trợ
Unlabeled rows saved in df_remain.xlsx
